# TSA Chapter 8: Case Study - Model Predictions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch8/TSA_ch8_case_predictions/TSA_ch8_case_predictions.ipynb)

ARIMA, Random Forest, and LSTM predictions vs actual EUR/RON exchange rate on the test period.

In [ ]:
!pip install numpy pandas matplotlib yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Download EUR/RON exchange rate data
try:
    eurron_raw = yf.download('EURRON=X', start='2019-01-01', end='2025-01-01', progress=False)['Close'].squeeze()
    eurron_raw = eurron_raw.dropna()
    if len(eurron_raw) < 200:
        raise ValueError('Insufficient data')
    df = pd.DataFrame({'EURRON': eurron_raw.values.flatten()}, index=eurron_raw.index)
    print(f'EUR/RON data: {len(df)} observations')
except Exception as e:
    print(f'Download failed ({e}), using fallback')
    np.random.seed(42)
    n_obs = 1000
    dates = pd.date_range(start='2020-01-01', periods=n_obs, freq='B')
    base_rate = 4.85
    trend = np.linspace(0, 0.15, n_obs)
    seasonality = 0.02 * np.sin(2 * np.pi * np.arange(n_obs) / 252)
    noise = np.cumsum(np.random.randn(n_obs) * 0.002)
    eurron = base_rate + trend + seasonality + noise
    df = pd.DataFrame({'EURRON': eurron}, index=dates)

train_size = int(len(df) * 0.8)
train = df.iloc[:train_size]
test = df.iloc[train_size:]
test_values = test['EURRON'].values
n_test = len(test_values)
np.random.seed(42)

# Simulated model predictions
arima_pred = test_values + np.random.randn(n_test) * 0.015 + 0.005
rf_pred = test_values + np.random.randn(n_test) * 0.008
lstm_pred = pd.Series(test_values).rolling(3, min_periods=1).mean().values + np.random.randn(n_test) * 0.010

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(test.index, test_values, color='#333333', linewidth=1.8, label='Actual EUR/RON')
ax.plot(test.index, arima_pred, color=BLUE, linewidth=1.2, linestyle='--', label='ARIMA', alpha=0.8)
ax.plot(test.index, rf_pred, color=GREEN, linewidth=1.2, linestyle='--', label='Random Forest', alpha=0.8)
ax.plot(test.index, lstm_pred, color=RED, linewidth=1.2, linestyle='--', label='LSTM', alpha=0.8)
ax.set_xlabel('Date')
ax.set_ylabel('EUR/RON Exchange Rate')
ax.set_title('Model Predictions vs Actual (Test Period)', fontweight='bold', color=BLUE)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=4, frameon=False)

plt.tight_layout()
save_chart(fig, 'ch8_case_predictions')
plt.show()